In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
from pyspark.sql.functions import when, col, rank, dense_rank, round, count, sum, avg, min, max
from pyspark.sql.window import Window

In [0]:
maha_results_df = spark.read.format('csv').option('header', 'true').option('inferSchema', 'true').load("dbfs:/FileStore/maha_results_2024.csv")
maha_results_df.show(5)

+-----+---------+-----+--------------------+--------------------+---------+------------+-----------+--------------+
|AC No|  AC Name|Sl no|           Candidate|               Party|EVM Votes|Postal Votes|Total Votes|Vote Share (%)|
+-----+---------+-----+--------------------+--------------------+---------+------------+-----------+--------------+
|    1|AKKALKUWA|    1|  AMSHYA FULJI PADVI|           Shiv Sena|    72411|         218|      72629|         31.55|
|    1|AKKALKUWA|    2|      ADV K C PADAVI|Indian National C...|    69122|         603|      69725|         30.29|
|    1|AKKALKUWA|    3|PADMAKAR VIJAYSIN...|Bharat Adivasi Party|     8640|         192|       8832|          3.84|
|    1|AKKALKUWA|    4|ENG. JELSING BIJA...|         Independent|     2895|          16|       2911|          1.26|
|    1|AKKALKUWA|    5|  SARYA DHARMA PADVI|         Independent|     1466|           7|       1473|          0.64|
+-----+---------+-----+--------------------+--------------------+-------

In [0]:
# Renaming All the Column names to Easy Read
maha_results_df = maha_results_df.withColumnRenamed("AC No", "Constituency_Number").withColumnRenamed("AC Name", "Constituency_Name").withColumnRenamed("Sl no", "Serial_Number").withColumnRenamed("Candidate", "Candidate_Name").withColumnRenamed("EVM Votes", "EVM_Votes").withColumnRenamed("Postal Votes", "Postal_Votes").withColumnRenamed("Total Votes", "Total_Votes").withColumnRenamed("Vote Share (%)", "Vote_Share_in_Percent")

In [0]:
maha_results_df.printSchema()

root
 |-- Constituency_Number: integer (nullable = true)
 |-- Constituency_Name: string (nullable = true)
 |-- Serial_Number: integer (nullable = true)
 |-- Candidate_Name: string (nullable = true)
 |-- Party: string (nullable = true)
 |-- EVM_Votes: integer (nullable = true)
 |-- Postal_Votes: integer (nullable = true)
 |-- Total_Votes: integer (nullable = true)
 |-- Vote_Share_in_Percent: double (nullable = true)



In [0]:
maha_results_df.show(5)

+-------------------+-----------------+-------------+--------------------+--------------------+---------+------------+-----------+---------------------+
|Constituency_Number|Constituency_Name|Serial_Number|      Candidate_Name|               Party|EVM_Votes|Postal_Votes|Total_Votes|Vote_Share_in_Percent|
+-------------------+-----------------+-------------+--------------------+--------------------+---------+------------+-----------+---------------------+
|                  1|        AKKALKUWA|            1|  AMSHYA FULJI PADVI|           Shiv Sena|    72411|         218|      72629|                31.55|
|                  1|        AKKALKUWA|            2|      ADV K C PADAVI|Indian National C...|    69122|         603|      69725|                30.29|
|                  1|        AKKALKUWA|            3|PADMAKAR VIJAYSIN...|Bharat Adivasi Party|     8640|         192|       8832|                 3.84|
|                  1|        AKKALKUWA|            4|ENG. JELSING BIJA...|        

In [0]:
#1.	Count the Total Constituencies: Find the total number of constituencies in the dataset
maha_results_df.select("Constituency_Number").distinct().count()

Out[7]: 288

In [0]:
#2.	List All Parties: Retrieve a distinct list of all parties (party) in the dataset.
maha_results_df.select("Party").distinct().count()

Out[8]: 160

In [0]:
#3.	Candidate with the Highest Votes in a Constituency: Display the candidate with the highest Total_Votes for each Constituency_Number.
windowSpec = Window.partitionBy("Constituency_Number").orderBy(col("Total_Votes").desc())

ranked_df = maha_results_df.withColumn("Rank", rank().over(windowSpec))
ranked_df.filter(col("Rank") == 1).show()

+-------------------+-----------------+-------------+--------------------+--------------------+---------+------------+-----------+---------------------+----+
|Constituency_Number|Constituency_Name|Serial_Number|      Candidate_Name|               Party|EVM_Votes|Postal_Votes|Total_Votes|Vote_Share_in_Percent|Rank|
+-------------------+-----------------+-------------+--------------------+--------------------+---------+------------+-----------+---------------------+----+
|                  1|        AKKALKUWA|            1|  AMSHYA FULJI PADVI|           Shiv Sena|    72411|         218|      72629|                31.55|   1|
|                  2|          SHAHADA|            1|RAJESH UDESING PADVI|Bharatiya Janata ...|   145660|        1179|     146839|                59.86|   1|
|                  3|        NANDURBAR|            3|DR.VIJAYKUMAR KRU...|Bharatiya Janata ...|   154025|        1165|     155190|                64.62|   1|
|                  4|          NAWAPUR|            2

In [0]:
#4.	Average Vote Share: Calculate the average Vote_Share_in_percent across all candidates.
maha_results_df.agg(avg("Vote_Share_in_Percent")).alias("Average_vote_share_percentage").show()

+--------------------------+
|avg(Vote_Share_in_Percent)|
+--------------------------+
|         6.509916365280306|
+--------------------------+



In [0]:
#5.	Total Postal Votes by Party: Summarize the total Postal_Votes for each party.
maha_results_df.groupBy('Party').agg(sum(col('Postal_Votes')).alias("Total_Postal_Votes")).orderBy(col("Total_Postal_Votes").desc()).show()

+--------------------+------------------+
|               Party|Total_Postal_Votes|
+--------------------+------------------+
|Bharatiya Janata ...|            123098|
|Indian National C...|             84348|
|Nationalist Congr...|             73098|
|Shiv Sena (Uddhav...|             67184|
|           Shiv Sena|             57452|
|Nationalist Congr...|             42655|
|         Independent|             37308|
|Vanchit Bahujan  ...|             12680|
|Maharashtra Navni...|              5827|
|   None of the Above|              4872|
|Peasants And Work...|              3437|
| Bahujan Samaj Party|              3388|
|  Jan Surajya Shakti|              3327|
|Prahar Janshakti ...|              3185|
|Rashtriya Samaj P...|              2941|
|All India Majlis-...|              2489|
|Rashtriya Yuva Sw...|              1683|
|Communist Party o...|              1427|
|Bahujan Vikas Aag...|              1053|
|Rajarshi Shahu Vi...|               899|
+--------------------+------------

In [0]:
# 6.	Winning Candidate Per Constituency: Identify the candidate with the maximum Total_Votes in each constituency.
windowSpec = Window.partitionBy("Constituency_Number").orderBy(col("Total_Votes").desc())

max_total_df = maha_results_df.withColumn("Rank", rank().over(windowSpec))
max_total_df.filter(col("Rank") == 1).show()

+-------------------+-----------------+-------------+--------------------+--------------------+---------+------------+-----------+---------------------+----+
|Constituency_Number|Constituency_Name|Serial_Number|      Candidate_Name|               Party|EVM_Votes|Postal_Votes|Total_Votes|Vote_Share_in_Percent|Rank|
+-------------------+-----------------+-------------+--------------------+--------------------+---------+------------+-----------+---------------------+----+
|                  1|        AKKALKUWA|            1|  AMSHYA FULJI PADVI|           Shiv Sena|    72411|         218|      72629|                31.55|   1|
|                  2|          SHAHADA|            1|RAJESH UDESING PADVI|Bharatiya Janata ...|   145660|        1179|     146839|                59.86|   1|
|                  3|        NANDURBAR|            3|DR.VIJAYKUMAR KRU...|Bharatiya Janata ...|   154025|        1165|     155190|                64.62|   1|
|                  4|          NAWAPUR|            2

In [0]:
#   7.	Votes from EVM: Show the total EVM_Votes cast across all constituencies.
maha_results_df.agg(sum("EVM_Votes").alias("Total_EVM_Votes")).show()


+---------------+
|Total_EVM_Votes|
+---------------+
|       64054283|
+---------------+



In [0]:

#   8.	Vote Share Above 50%: List candidates who have a Vote_Share_in_percent greater than 50.
maha_results_df.filter(col("Vote_Share_in_Percent") > 50)


Out[14]: DataFrame[Constituency_Number: int, Constituency_Name: string, Serial_Number: int, Candidate_Name: string, Party: string, EVM_Votes: int, Postal_Votes: int, Total_Votes: int, Vote_Share_in_Percent: double]

In [0]:

#   9.	Constituencies with No Postal Votes: Find constituencies where Postal_Votes are zero.
maha_results_df.select("Constituency_Number", "Constituency_Name").filter(col("Postal_Votes") == 0).distinct().show()

+-------------------+------------------+
|Constituency_Number| Constituency_Name|
+-------------------+------------------+
|                 26|          KHAMGAON|
|                 36|DHAMANGAON RAILWAY|
|                  6|       DHULE RURAL|
|                 31|        AKOLA EAST|
|                 25|            MEHKAR|
|                 13|      JALGAON CITY|
|                 30|        AKOLA WEST|
|                 24|     SINDKHED RAJA|
|                 20|       MUKTAINAGAR|
|                 23|           CHIKHLI|
|                 29|           BALAPUR|
|                 21|          MALKAPUR|
|                 33|             RISOD|
|                 14|     JALGAON RURAL|
|                 15|           AMALNER|
|                  8|         SINDKHEDA|
|                 35|           KARANJA|
|                 22|          BULDHANA|
|                 17|        CHALISGAON|
|                 37|           BADNERA|
+-------------------+------------------+
only showing top

In [0]:
#   10.	Count Candidates per Party: Display the total number of candidates for each party.
maha_results_df.groupBy("Party").count().orderBy(col("count").desc()).show()

+--------------------+-----+
|               Party|count|
+--------------------+-----+
|         Independent| 2086|
|   None of the Above|  288|
| Bahujan Samaj Party|  237|
|Vanchit Bahujan  ...|  200|
|Bharatiya Janata ...|  149|
|Maharashtra Navni...|  125|
|Indian National C...|  101|
|Shiv Sena (Uddhav...|   95|
|Rashtriya Samaj P...|   93|
|Nationalist Congr...|   86|
|           Shiv Sena|   81|
|Nationalist Congr...|   59|
|Peoples Party of ...|   44|
|Prahar Janshakti ...|   38|
|Maharashtra Swara...|   32|
|Republican Party ...|   31|
|Aazad Samaj Party...|   28|
|Bahujan Republica...|   22|
|     Republican Sena|   21|
|   Swabhimani Paksha|   19|
+--------------------+-----+
only showing top 20 rows



In [0]:
#   11.	Top 3 Parties Based on Total Votes:
#   Retrieve the top 3 parties with the highest sum of Total_Votes.

maha_results_df.groupBy("Party").agg(sum("Total_Votes").alias("Total_Votes_Per_Party")).orderBy(col("Total_Votes_Per_Party").desc()).limit(3).show()

+--------------------+---------------------+
|               Party|Total_Votes_Per_Party|
+--------------------+---------------------+
|Bharatiya Janata ...|             17293650|
|Indian National C...|              8020921|
|           Shiv Sena|              7996930|
+--------------------+---------------------+



In [0]:
#   12. for each party from total_no_of_candidates how many candidates are won ?

# Define window for dense rank
windowSpec = Window.partitionBy("Constituency_Number").orderBy(col("Total_Votes").desc())

# Add dense rank column
candidate_ranks = maha_results_df.withColumn("rnk", dense_rank().over(windowSpec))

# Aggregate results: count total candidates and winning candidates
result = (
    candidate_ranks.groupBy("party")
    .agg(
        count("*").alias("total_no_of_candidates"),
        count(when(col("rnk") == 1, 1)).alias("no_of_winning_candidates")
    )
    .orderBy(col("no_of_winning_candidates").desc())
)

# Show the result
result.show()

+--------------------+----------------------+------------------------+
|               party|total_no_of_candidates|no_of_winning_candidates|
+--------------------+----------------------+------------------------+
|Bharatiya Janata ...|                   149|                     132|
|           Shiv Sena|                    81|                      57|
|Nationalist Congr...|                    59|                      41|
|Shiv Sena (Uddhav...|                    95|                      20|
|Indian National C...|                   101|                      16|
|Nationalist Congr...|                    86|                      10|
|  Jan Surajya Shakti|                     6|                       2|
|     Samajwadi Party|                     9|                       2|
|         Independent|                  2086|                       2|
|Rashtriya Samaj P...|                    93|                       1|
|Rashtriya Yuva Sw...|                     2|                       1|
|Commu

In [0]:
#13.   Constituencies with Closest Margins: Find constituencies where the difference in Total_Votes between the top two candidates is less than 100.

windowSpec = Window.partitionBy("Constituency_Number").orderBy(col("Total_Votes").desc())

ranked_candidates = maha_results_df.withColumn("rnk", dense_rank().over(windowSpec))

self_join_df = ranked_candidates.alias("rc1").join(
    ranked_candidates.alias("rc2"),
    (col("rc1.Constituency_Number") == col("rc2.Constituency_Number")) &
    (col("rc1.rnk") == 1) & (col("rc2.rnk") == 2),
    "inner"
).filter(col("rc1.Total_Votes") - col("rc2.Total_Votes") < 100).select(
    "rc1.Constituency_Number", "rc1.Constituency_Name"
)

self_join_df.show()

+-------------------+-----------------+
|Constituency_Number|Constituency_Name|
+-------------------+-----------------+
+-------------------+-----------------+



In [0]:
#   14.	Winning Margin for Each Constituency:s
#   Calculate the winning margin (difference in votes) for each constituency.

windowSpec = Window.partitionBy("Constituency_Number").orderBy(col("Total_Votes").desc())

ranked_candidates = maha_results_df.withColumn("rnk", dense_rank().over(windowSpec))

self_join_df = ranked_candidates.alias("rc1").join(
    ranked_candidates.alias("rc2"),
    (col("rc1.Constituency_Number") == col("rc2.Constituency_Number")) &
    (col("rc1.rnk") == 1) & (col("rc2.rnk") == 2),
    "inner"
).withColumn("Winning_margin", col("rc1.Total_Votes")-col("rc2.Total_Votes"))

self_join_df.select("rc1.Constituency_Number", "rc1.Constituency_Name", "Winning_margin").orderBy(col("Winning_margin").desc()).show()

+-------------------+-------------------+--------------+
|Constituency_Number|  Constituency_Name|Winning_margin|
+-------------------+-------------------+--------------+
|                  9|            SHIRPUR|        145944|
|                262|             SATARA|        142124|
|                233|              PARLI|        140224|
|                116|             BAGLAN|        129297|
|                219|          KOPARGAON|        124624|
|                147|KOPRI - PACHPAKHADI|        120717|
|                 54|        NAGPUR EAST|        115288|
|                210|            KOTHRUD|        112041|
|                204|              MAVAL|        108565|
|                146|   OVALA - MAJIWADA|        108158|
|                 41|            MELGHAT|        106859|
|                115|     MALEGAON OUTER|        106606|
|                205|          CHINCHWAD|        103865|
|                201|           BARAMATI|        100899|
|                152|          

In [0]:
#   15.	Highest Vote Share by Party:
#   Identify the constituency and candidate where each party received its highest Vote_Share_in_percent.

maha_results_df.groupBy("Party", "Constituency_Number", "Candidate_Name").agg(max("Vote_Share_in_Percent").alias("Maximum_vote_share")).show()

+--------------------+-------------------+--------------------+------------------+
|               Party|Constituency_Number|      Candidate_Name|Maximum_vote_share|
+--------------------+-------------------+--------------------+------------------+
|         Independent|                 14|SHIVAJI MAHARU HA...|              0.16|
|         Independent|                 35|VIJAY BAJIRAO WAN...|              0.07|
|         Independent|                 85|MADHAV NARSING ME...|              0.02|
|         Independent|                100|         NISAR PATEL|              0.26|
| Bahujan Samaj Party|                101| KISHOR YADAV BORUDE|               0.4|
|         Independent|                103|KESHAV RAMKISAN D...|              0.62|
|   None of the Above|                114|                NOTA|              0.46|
|         Independent|                120|SAGAR PANDURANG S...|              0.93|
|Vanchit Bahujan  ...|                126|DR. AVINASH NIRAN...|              6.44|
|Ind

In [0]:
#   16.	Votes Percentage in Top 3 Constituencies:
#   For the top 3 constituencies with the highest total votes, calculate each candidate's percentage share of votes.

top_3_constituencies = maha_results_df.groupBy(
    "Constituency_Number", "Constituency_Name").agg(
        sum("Total_Votes").alias("Total_votes_per_constituency")
    ).orderBy(col("Total_votes_per_constituency").desc()).limit(3)

joined_df = (
    maha_results_df.alias("mr").join(
        top_3_constituencies.alias("tc"),
        col("mr.Constituency_Number") == col("tc.Constituency_Number"),
        "inner"
    ).withColumn("vote_percentage", round(col("mr.Total_Votes")*100 / col("tc.Total_votes_per_constituency"),2)).select(
        col("mr.Candidate_Name"), 
        col("mr.Party"), 
        col("mr.Constituency_Number"), 
        col("mr.Constituency_Name"), 
        col("mr.Total_Votes"), 
        col("vote_percentage")
    )
)

joined_df.show()

+--------------------+--------------------+-------------------+-----------------+-----------+---------------+
|      Candidate_Name|               Party|Constituency_Number|Constituency_Name|Total_Votes|vote_percentage|
+--------------------+--------------------+-------------------+-----------------+-----------+---------------+
|GAJENDRA KRUSHNAD...| Bahujan Samaj Party|                188|           PANVEL|       1526|            0.4|
|PRASHANT RAMSHETH...|Bharatiya Janata ...|                188|           PANVEL|     183931|           47.9|
|YOGESH JANARDAN C...|Maharashtra Navni...|                188|           PANVEL|      10231|           2.66|
|   LEENA ARJUN GARAD|Shiv Sena (Uddhav...|                188|           PANVEL|      43989|          11.46|
|KANTILAL HARISHCH...|Lokmudra Janhit P...|                188|           PANVEL|       2429|           0.63|
| PAVAN UTTAMRAO KALE|Bhartiya Jan Samr...|                188|           PANVEL|        477|           0.12|
|BALARAM D

In [0]:
#   17.	Party-Wise Vote Share Distribution:
#   Calculate the average Vote_Share_in_percent for each party.
maha_results_df.groupBy("Party").agg(round(avg("Vote_Share_in_Percent"),2).alias("Average_Vote_Share")).show()

+--------------------+------------------+
|               Party|Average_Vote_Share|
+--------------------+------------------+
|Netaji Congress Sena|              0.05|
|   None of the Above|              0.75|
|Janata Dal  (Secu...|              0.15|
|Swarajya Rashtra ...|              0.25|
|Socialist Unity C...|              0.13|
|Sambhaji Brigade ...|              0.32|
|Nationalist Congr...|             42.07|
|Ambedkarist Repub...|              0.07|
|Social Democratic...|               0.3|
|Digital Organizat...|              0.27|
|Bhartiya Navjawan...|              0.04|
|     Janata Congress|              0.06|
|Rashtriya Samaj P...|              1.02|
|Bhartiya Asmita P...|              0.14|
|Rashtriya Yuva Sw...|             46.39|
|Revolutionary Mar...|               0.4|
|All India Forward...|              0.07|
|Bhartiya Tribal P...|              0.32|
|Right to Recall P...|              0.17|
|Bahujan Samaj Par...|              0.33|
+--------------------+------------

In [0]:
#   18.	Identify Constituencies with Most Candidates:
#   Find the constituencies with the highest number of candidates.

maha_results_df.groupBy(
    "Constituency_Number", 
    "Constituency_Name"
).agg(count("Candidate_Name").alias("No_of_Candidates")).orderBy(col("No_of_Candidates").desc()).show()

+-------------------+------------------+----------------+
|Constituency_Number| Constituency_Name|No_of_Candidates|
+-------------------+------------------+----------------+
|                229|         MAJALGAON|              35|
|                 86|      NANDED NORTH|              34|
|                103|         BHOKARDAN|              33|
|                230|              BEED|              32|
|                 13|      JALGAON CITY|              30|
|                109|   AURANGABAD EAST|              30|
|                106|         PHULAMBRI|              28|
|                 35|           KARANJA|              28|
|                101|             JALNA|              27|
|                 57|      NAGPUR NORTH|              27|
|                 37|           BADNERA|              27|
|                232|              KAIJ|              26|
|                251|     SOLAPUR SOUTH|              26|
|                 85|            BHOKAR|              26|
|             

In [0]:
#   19.	Postal Votes Contribution by Party:
#   Calculate the percentage contribution of Postal_Votes for each party.

maha_results_df.groupBy(
    "Party"
).agg(
    round(sum("Postal_Votes")*100 / sum("Total_Votes"),2).alias("contribution_of_postal_votes")
).show()


+--------------------+----------------------------+
|               Party|contribution_of_postal_votes|
+--------------------+----------------------------+
|Netaji Congress Sena|                         0.0|
|   None of the Above|                        1.05|
|Janata Dal  (Secu...|                         0.3|
|Swarajya Rashtra ...|                        0.93|
|Socialist Unity C...|                         0.0|
|Sambhaji Brigade ...|                         0.5|
|Nationalist Congr...|                        0.73|
|Ambedkarist Repub...|                         1.3|
|Social Democratic...|                        0.38|
|Digital Organizat...|                        1.35|
|Bhartiya Navjawan...|                        1.74|
|     Janata Congress|                        0.53|
|Rashtriya Samaj P...|                         1.1|
|Bhartiya Asmita P...|                        0.65|
|Rashtriya Yuva Sw...|                        0.86|
|Revolutionary Mar...|                        0.21|
|All India F

In [0]:
#   20.	Filter Constituencies by Party Performance:
#   List constituencies where a specific party won (e.g., party = 'XYZ').

# Step 1: Calculate max total votes per constituencies
highest_votes_df = maha_results_df.groupBy("Constituency_Number").agg(max("Total_Votes").alias("Max_total_votes_per_constituency"))

# Step 2: Join with the original DataFrame
result_df = highest_votes_df.alias("hv").join(
    maha_results_df.alias("mr"),
    (col("hv.Constituency_Number") == col("mr.Constituency_Number")) & 
    (col("Max_total_votes_per_constituency") == col("Total_Votes")),
    "inner"
).select(
    col("hv.Constituency_Number").alias("Constituency_Number"),
    "Constituency_Name",
    "Candidate_Name",
    col("mr.Total_Votes")
)

# Step 3: Show the result
result_df.show()

+-------------------+-----------------+--------------------+-----------+
|Constituency_Number|Constituency_Name|      Candidate_Name|Total_Votes|
+-------------------+-----------------+--------------------+-----------+
|                  1|        AKKALKUWA|  AMSHYA FULJI PADVI|      72629|
|                  2|          SHAHADA|RAJESH UDESING PADVI|     146839|
|                  3|        NANDURBAR|DR.VIJAYKUMAR KRU...|     155190|
|                  4|          NAWAPUR|SHIRISHKUMAR SURU...|      87166|
|                  5|            SAKRI|MANJULA TULSHIRAM...|     104649|
|                  6|      DHULE RURAL|RAGHAVENDRA (RAMD...|     170398|
|                  7|       DHULE CITY|AGRAWAL ANUPBHAIY...|     116538|
|                  8|        SINDKHEDA|JAYKUMAR JITENDRA...|     151492|
|                  9|          SHIRPUR|KASHIRAM VECHAN P...|     178073|
|                 10|           CHOPDA|CHANDRAKANT BALIR...|     122826|
|                 11|            RAVER|AMOL HARIBHA

In [0]:
#   21.	Party-Wise Top Performers:
#   Identify the top-performing candidate for each party.

# Step 1: Calculate max total votes per party
highest_votes_df = maha_results_df.groupBy("Party").agg(max("Total_Votes").alias("Max_total_votes_per_party"))

# Step 2: Join with the original DataFrame
result_df = highest_votes_df.alias("hv").join(
    maha_results_df.alias("mr"),
    (col("hv.Party") == col("mr.Party")) & 
    (col("Max_total_votes_per_party") == col("Total_Votes")),
    "inner"
).select(
    col("hv.Party").alias("Party"),
    "Constituency_Number",
    "Constituency_Name",
    "Candidate_Name"
)

# Step 3: Show the result
result_df.show()

+--------------------+-------------------+------------------+--------------------+
|               Party|Constituency_Number| Constituency_Name|      Candidate_Name|
+--------------------+-------------------+------------------+--------------------+
|Bharat Adivasi Party|                  1|         AKKALKUWA|PADMAKAR VIJAYSIN...|
|Minorities Democr...|                  8|         SINDKHEDA| GULAM AYYUB PINJARI|
|Communist Party o...|                  9|           SHIRPUR|   BUDHA MALA PAWARA|
|   Vikas India Party|                 13|      JALGAON CITY|MAMTA BHIKARI TAD...|
|Rashtriya Janmanc...|                 17|        CHALISGAON| SANDIP ASHOK LANDGE|
|Socialist Party (...|                 23|           CHIKHLI|MACCHINDRA SHESHR...|
|Jantantrik Samta ...|                 26|          KHAMGAON|PAVAN KESHAV JAIN...|
|Indian National L...|                 26|          KHAMGAON|SHAIKH RASHID SHA...|
|Maharashtra Swara...|                 27|   JALGAON (JAMOD)|PRASHANT KASHIRAM...|
|Van